In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
np.random.seed(4590)

from scipy import stats
from scipy.stats import pearsonr


from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# %%time
# Path = '../input/elo-ref-2-data-conversion/'
# historical_transactions = pd.read_hdf(Path+'historical_transactions.hdf')
# print('hist transactions read complete')
# new_transactions = pd.read_hdf(Path+'new_transactions.hdf')
# print('new transactions read complete')

In [4]:
%%time
Path = '../input/elo-ref-2-data-conversion/'
historical_transactions = pd.read_csv(Path+'historical_transactions.csv',index_col=0)
print('hist transactions read complete')
new_transactions = pd.read_csv(Path+'new_transactions.csv',index_col=0)
print('new transactions read complete')

hist transactions read complete
new transactions read complete
CPU times: user 1min 10s, sys: 13 s, total: 1min 23s
Wall time: 1min 23s


In [5]:
historical_transactions= reduce_mem_usage(historical_transactions)
new_transactions= reduce_mem_usage(new_transactions)

Mem. usage decreased to 1582.53 Mb (52.5% reduction)
Mem. usage decreased to 102.97 Mb (54.2% reduction)


In [6]:
# cardid ='C_ID_749cfc0c3a'
# new_transactions[historical_transactions['card_id']==cardid]

In [7]:
# %%time
# train = pd.read_hdf('../input/elo-preproc-3/train_preproc.hdf')
# test = pd.read_hdf('../input/elo-preproc-3/test_preproc.hdf')

In [8]:
%%time
train = pd.read_csv('../input/elo-preproc-3/train_preproc.csv',index_col=0)
test = pd.read_csv('../input/elo-preproc-3/test_preproc.csv',index_col=0)

CPU times: user 8 s, sys: 160 ms, total: 8.16 s
Wall time: 8.17 s


In [9]:
# pearsonr(train['new_hist_category_1_mean'].fillna(-10000),train['hist_category_1_mean'])

In [10]:
mask = train['target'] < -30
train['outliers'] = 0
train.loc[mask,'outliers'] = 1

In [11]:
np.round(np.array([-0.737793,-0.491455,0.602539,0.831055]) / 0.00150265118 + 497.06,2)

array([   6.07,  170.  ,  898.04, 1050.12])

In [12]:
new_transactions['purchase_amount'] = new_transactions['purchase_amount'].astype('float32')
new_transactions['purchase_amount_new'] = np.round(new_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
historical_transactions['purchase_amount_new'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06,2)
# train['target_raw'] = 2**train['target']

In [13]:
# historical_transactions = pd.merge(historical_transactions,train[['target','outliers','card_id']],on='card_id',how='left')
# print('history merge complete')

# new_transactions = pd.merge(new_transactions,train[['target','outliers','card_id']],on='card_id',how='left')
# print('new merge complete')


In [14]:
print(new_transactions['month_lag'].unique())
historical_transactions['month_lag'].unique()

[1 2]


array([ -8,  -7,  -6,  -5, -11,   0,  -3,  -9,  -4,  -1, -13, -10, -12,
        -2])

In [15]:
# # scaler = MinMaxScaler(feature_range=(1,2))
# # scaler.fit(historical_transactions['purchase_amount'].values.reshape(-1,1))
# # scaled= scaler.transform(historical_transactions['purchase_amount'].values.reshape(-1,1))
# for df in [historical_transactions,new_transactions]:
#     mask = (df['installments']==-1) | (df['installments']==999)
#     df.loc[mask,'purchase_amount_per_install'] = df[mask]['purchase_amount_new'] 
#     df.loc[~mask,'purchase_amount_per_install'] = df[~mask]['purchase_amount_new'] /  (df[~mask]['installments'] + 1)
#     print(df['purchase_amount_per_install'].describe())

In [16]:
hist_gp = historical_transactions.groupby(['card_id','month_lag']).agg({'purchase_amount_new':'mean'}).reset_index()
new_gp = new_transactions.groupby(['card_id','month_lag']).agg({'purchase_amount_new':'mean',}).reset_index()

In [17]:
# hist_gp = historical_transactions.groupby(['card_id','month_lag']).agg({'purchase_amount_new':'sum','target': 'first','outliers':'first'}).reset_index()
# new_gp = new_transactions.groupby(['card_id','month_lag']).agg({'purchase_amount_new':'sum','target':'first','outliers':'first'}).reset_index()

In [18]:
def filter_monthlag(cardids,trans,month_lag,purch_col='purchase_amount_new'):
    trans_filt = trans[trans['month_lag']==month_lag]
    missed_cardids=set(cardids.unique()).difference(set(trans_filt['card_id'].unique())) 
    miss_df = pd.DataFrame()
    miss_df['card_id'] = np.array(list(missed_cardids))
    miss_df['month_lag'] = 1000
    miss_df[purch_col] = 0
    trans_filt =pd.concat([trans_filt,miss_df],axis=0)
    trans_filt.sort_values('card_id',inplace=True)
    trans_filt.reset_index(inplace=True)
    return trans_filt

def filter_monthlag_max(cardids,trans,month_lag,tofilter=True,purch_col='purchase_amount_new'):
    if tofilter:
        trans_filt = trans[trans['month_lag']<month_lag]
    else:
        trans_filt = trans
        
#     trans_filt = trans_filt.loc[trans_filt.groupby('card_id')['month_lag'].idxmax()]
    
    trans_filt['index_orig'] = trans_filt.index
    trans_filt = trans_filt.sort_values("month_lag",ascending=False).groupby("card_id", as_index=False).first()
    missed_cardids=set(cardids.unique()).difference(set(trans_filt['card_id'].unique())) 
    miss_df = pd.DataFrame()
    miss_df['card_id'] = np.array(list(missed_cardids))
    miss_df['month_lag'] = 1000
    miss_df[purch_col] = 0
    miss_df.index = np.repeat([1E+20],miss_df.shape[0]) 
    print('miss_df.shape:',miss_df.shape)
#     print('miss_df.index:',miss_df.index)
    trans_filt =pd.concat([trans_filt,miss_df],axis=0)
    trans_filt.sort_values('card_id',inplace=True)
#     print('trans_filt.index:',trans_filt.index)
    trans_filt.reset_index(inplace=True)
    print('trans filt shape final:',trans_filt.shape)
#     print('trans_filt[index] after:',trans_filt['index'].nunique())
    return trans_filt

def gen_lagratio(cardids,numerator,denominator,newcolname,train,test):
    ratio_df = pd.DataFrame()
    ratio_df['card_id'] = cardids

    eps = 1E-10
    ratio_df[newcolname] = numerator / (denominator + eps)
    ratio_df.loc[numerator==0,newcolname] = 0

    #merge with train and test
    train = pd.merge(train, ratio_df, on='card_id', how='left')
    test = pd.merge(test, ratio_df, on='card_id', how='left')
    
    return train,test

In [19]:
hist_gp.sort_values('card_id').reset_index(inplace=True)
new_gp.sort_values('card_id').reset_index(inplace=True)

In [20]:
# lagratiocols = [col for col in train.columns if ('lag_' in col) and (('_ratio') in col)]
# train.drop(lagratiocols,inplace=True,axis=1)
# test.drop(lagratiocols,inplace=True,axis=1)

In [21]:
purch_col ='purchase_amount_new'

In [22]:
# Lag 0 / Lag next ratio
month_lag =0
hist_gp_lag_0 = filter_monthlag(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)
hist_gp_lag_next = filter_monthlag_max(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)

train,test = gen_lagratio(hist_gp_lag_0['card_id'],hist_gp_lag_0[purch_col],
             hist_gp_lag_next[purch_col],'lag_avg_0_ratio',train,test)


miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)


In [23]:
# Lag -1 / Lag next ratio
month_lag = -1
hist_gp_lag_m_1 = filter_monthlag(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)
hist_gp_lag_m_1_next = filter_monthlag_max(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)

train,test = gen_lagratio(hist_gp_lag_m_1['card_id'],hist_gp_lag_m_1[purch_col],
             hist_gp_lag_m_1_next[purch_col],'lag_avg_-1_ratio',train,test)


miss_df.shape: (1560, 3)
trans filt shape final: (325540, 5)


In [24]:
#Lag 0  and Lag -1 ratio
# For second maximum , remove records by using drop (first max) and then generate maximum again
month_lag =-1
hist_gp_lag_m_1_next_2 = filter_monthlag_max(hist_gp['card_id'],
                                             hist_gp.drop(index=hist_gp_lag_m_1_next['index_orig'],errors='ignore',axis=0),
                                             month_lag,tofilter=False,purch_col=purch_col)
numerator = hist_gp_lag_0[purch_col] + hist_gp_lag_m_1[purch_col]
denominator = (hist_gp_lag_m_1_next[purch_col] + hist_gp_lag_m_1_next_2[purch_col] )

train,test = gen_lagratio(hist_gp_lag_m_1_next_2['card_id'],numerator,
             denominator,'lag_avg_0_2m_ratio',train,test)


miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)


In [25]:
#Lag 0  and Lag -1 and Lag -2 ratio

month_lag = -2
hist_gp_lag_m_2 = filter_monthlag(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)
hist_gp_lag_m_2_next = filter_monthlag_max(hist_gp['card_id'],hist_gp,month_lag,purch_col=purch_col)
idx1 = hist_gp_lag_m_2_next['index_orig']

hist_gp_lag_m_2_next_2 = filter_monthlag_max(hist_gp['card_id'],
                                             hist_gp.drop(index=idx1,axis=0,errors='ignore',),
                                             month_lag,tofilter=False,purch_col=purch_col)
idx2 = hist_gp_lag_m_2_next_2['index_orig']
idx_comb = pd.concat([idx1,idx2])

# idx_comb = set(list(idx1)).union(set(list(idx1)))
hist_gp_lag_m_2_next_3 = filter_monthlag_max(hist_gp['card_id'],
                                             hist_gp.drop(index=idx_comb,axis=0,errors='ignore',),
                                             month_lag,tofilter=False,purch_col=purch_col)

numerator = hist_gp_lag_0[purch_col] + hist_gp_lag_m_1[purch_col] + \
            hist_gp_lag_m_2[purch_col]
denominator = (hist_gp_lag_m_2_next[purch_col] +\
               hist_gp_lag_m_2_next_2[purch_col] +\
               hist_gp_lag_m_2_next_3[purch_col] )

train,test = gen_lagratio(hist_gp_lag_m_2['card_id'],numerator,
             denominator,'lag_avg_0_3m_ratio',train,test)


miss_df.shape: (25718, 3)
trans filt shape final: (325540, 5)
miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)
miss_df.shape: (96, 3)
trans filt shape final: (325540, 5)


In [26]:
#New transactions as future trans
new_gp_lag_2 = filter_monthlag(hist_gp['card_id'],new_gp,2,purch_col=purch_col)
new_gp_lag_1 = filter_monthlag(hist_gp['card_id'],new_gp,1,purch_col=purch_col)

numerator = new_gp_lag_2[purch_col] 
denominator = new_gp_lag_1[purch_col] 
train,test = gen_lagratio(new_gp_lag_2['card_id'],numerator,
             denominator,'lag_avg_new_ratio',train,test)

In [27]:
#New transactions as future trans
# new_gp_lag_2 = filter_monthlag(hist_gp['card_id'],new_gp,2)
# new_gp_lag_1 = filter_monthlag(hist_gp['card_id'],new_gp,1)

hist_gp_lag_0_next = filter_monthlag_max(hist_gp['card_id'],hist_gp,1,purch_col=purch_col)
hist_gp_lag_0_next_2 = filter_monthlag_max(hist_gp['card_id'],
                                             hist_gp.drop(index=hist_gp_lag_0_next['index_orig'],errors='ignore',axis=0),
                                             0,tofilter=False,purch_col=purch_col)

numerator = new_gp_lag_2[purch_col] + new_gp_lag_1[purch_col]
denominator = (hist_gp_lag_0_next[purch_col] + hist_gp_lag_0_next_2[purch_col] )

train,test = gen_lagratio(new_gp_lag_2['card_id'],numerator,
             denominator,'lag_avg_new_2m_ratio',train,test)

miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)
miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)


In [28]:
#New transactions as future trans
# new_gp_lag_2 = filter_monthlag(hist_gp['card_id'],new_gp,2)
# new_gp_lag_1 = filter_monthlag(hist_gp['card_id'],new_gp,1)

hist_gp_lag_0_next = filter_monthlag_max(hist_gp['card_id'],hist_gp,1,purch_col=purch_col)
hist_gp_lag_0_next_2 = filter_monthlag_max(hist_gp['card_id'],
                                             hist_gp.drop(index=hist_gp_lag_0_next['index_orig'],errors='ignore',axis=0),
                                             0,tofilter=False,purch_col=purch_col)

numerator =(new_gp_lag_2[purch_col] + new_gp_lag_1[purch_col])
denominator =  (hist_gp_lag_0_next[purch_col]+hist_gp_lag_0_next_2[purch_col]) / 2

train,test = gen_lagratio(new_gp_lag_2['card_id'],numerator,
             denominator,'lag_avg_new_2m_avg_ratio',train,test)

miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)
miss_df.shape: (0, 3)
trans filt shape final: (325540, 5)


In [29]:
#New transactions as future trans
numerator = new_gp_lag_1[purch_col] 
denominator = hist_gp_lag_0_next[purch_col] 
train,test = gen_lagratio(new_gp_lag_2['card_id'],numerator,
             denominator,'lag_avg_new_1_to_0_ratio',train,test)

In [30]:
#New transactions as future trans
numerator = new_gp_lag_2[purch_col] 
denominator = hist_gp_lag_0_next[purch_col] 
train,test = gen_lagratio(new_gp_lag_2['card_id'],numerator,
             denominator,'lag_avg_new_2_to_0_ratio',train,test)

In [31]:
train['target_raw'] = 2**train['target']

In [32]:
ratiocols = ['lag_avg_new_ratio','lag_avg_new_2m_ratio','lag_avg_new_1_to_0_ratio','lag_avg_new_2_to_0_ratio',
#              'lag_new_2m_avg_ratio',
             'lag_avg_-1_ratio','lag_avg_0_ratio','lag_avg_0_3m_ratio','lag_avg_0_2m_ratio']
# ratiocols = ['lag_new_2m_ratio']
for col in ratiocols:
    coeff,pval= stats.pearsonr(train[col].values,
                           train['target'].values)
    print('{0} correlation:{1} pval: {2} '.format(col,coeff,pval))
    
# coeff,pval= stats.pearsonr(train[ratiocols].mean(axis=1).values,
#                        train['target_raw'].values)
col = 'hist_month_nunique'
coeff,pval= stats.pearsonr(train[col].values,
                       train['target'].values)
print('{0} correlation:{1} pval: {2} '.format(col,coeff,pval))
    

lag_avg_new_ratio correlation:-0.01558785635249433 pval: 2.4731630308077357e-12 
lag_avg_new_2m_ratio correlation:-0.049613152605940423 pval: 3.1348491572185834e-110 
lag_avg_new_1_to_0_ratio correlation:-0.0031085230133148347 pval: 0.16246921052683105 
lag_avg_new_2_to_0_ratio correlation:9.442899497129436e-05 pval: 0.9661546600181022 
lag_avg_-1_ratio correlation:0.0031881377290726184 pval: 0.1519750251110043 
lag_avg_0_ratio correlation:0.0003785527724988443 pval: 0.8649296801605051 
lag_avg_0_3m_ratio correlation:0.0005855441034103025 pval: 0.7924629722567607 
lag_avg_0_2m_ratio correlation:0.0007824465115873083 pval: 0.7251445727653929 
hist_month_nunique correlation:-0.019252469457306354 pval: 5.065825334551969e-18 


In [33]:
del hist_gp,new_gp
del hist_gp_lag_0_next, hist_gp_lag_0_next_2, numerator, denominator
del new_gp_lag_1, new_gp_lag_2
del hist_gp_lag_0,hist_gp_lag_next
del hist_gp_lag_m_1,hist_gp_lag_m_1_next,hist_gp_lag_m_1_next_2
del hist_gp_lag_m_2,hist_gp_lag_m_2_next,hist_gp_lag_m_2_next_3
gc.collect()

308

In [34]:
# scaler = MinMaxScaler(feature_range=(1,2))
# scaler.fit(historical_transactions['purchase_amount'].values.reshape(-1,1))
# scaled= scaler.transform(historical_transactions['purchase_amount'].values.reshape(-1,1))
for df in [historical_transactions,new_transactions]:
    min_amount = df['purchase_amount'].min()
    df['purchase_amount_scaled'] = 100 * (df['purchase_amount'] - min_amount + 1)
    mask = (df['installments']==-1) | (df['installments']==999)
    df.loc[mask,'purchase_amount_per_install'] = df[mask]['purchase_amount_scaled'] 
    df.loc[~mask,'purchase_amount_per_install'] = df[~mask]['purchase_amount_scaled'] /  (df[~mask]['installments'] + 1)
    print(df['purchase_amount_scaled'].describe())
    print(df['purchase_amount_per_install'].describe())

count    2.911236e+07
mean     1.370152e+02
std      1.123521e+05
min      1.000000e+02
25%      1.026552e+02
50%      1.058558e+02
75%      1.143653e+02
max      6.010605e+08
Name: purchase_amount_scaled, dtype: float64
count    2.911236e+07
mean     1.422354e+02
std      1.122055e+05
min      7.698087e+00
25%      5.299028e+01
50%      1.009767e+02
75%      1.052593e+02
max      6.010605e+08
Name: purchase_amount_per_install, dtype: float64
count    1.963031e+06
mean     1.194826e+02
std      6.926756e+01
min      1.000000e+02
25%      1.030273e+02
50%      1.072266e+02
75%      1.165527e+02
max      2.649971e+04
Name: purchase_amount_scaled, dtype: float64
count    1.963031e+06
mean     8.482602e+01
std      4.987085e+01
min      8.420974e+00
25%      5.292969e+01
50%      1.004395e+02
75%      1.058594e+02
max      9.874707e+03
Name: purchase_amount_per_install, dtype: float64


In [35]:
def aggregate_transactions(history,agg_func,groupbyflag,prefix):
    if groupbyflag=='card':
        group_cols =['card_id']
        newcoltemplate = 'card_'
    elif groupbyflag=='merchant':
        group_cols =['merchant_id']
        newcoltemplate = 'merchant_'
    elif groupbyflag=='both':
        group_cols =['card_id','merchant_id']
        newcoltemplate = 'card_merchant_'
    agg_history = history[group_cols + list(agg_func.keys())].groupby(group_cols).agg(agg_func)
    print('groupby complete')
    new_columns = get_new_columns(prefix,agg_func)
    agg_history.columns = new_columns
#     agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    print('reset index complete')
    #generate size of each card merchant combination
#     agg_history[newcoltemplate +'size']=history[group_cols].groupby(group_cols).size().reset_index()
    
    return agg_history
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]


In [36]:
# mask1 = historical_transactions['category_3']=='A'
# mask2 = historical_transactions['category_3']=='B'
# mask3 = historical_transactions['category_3']=='C'
# print(historical_transactions.loc[mask1,'card_id'].unique()[0:2])
# print(historical_transactions.loc[mask2,'card_id'].unique()[0:2])
# print(historical_transactions.loc[mask3,'card_id'].unique()[0:2])
# mask1 = new_transactions['category_3']=='A'
# mask2 = new_transactions['category_3']=='B'
# mask3 = new_transactions['category_3']=='C'
# print(new_transactions.loc[mask1,'card_id'].unique()[0:2])
# print(new_transactions.loc[mask1,'card_id'].unique()[0:2])
# print(new_transactions.loc[mask1,'card_id'].unique()[0:2])

In [37]:
#convert category 3 to numeric
for df in [historical_transactions,new_transactions]:
    df['category_3_raw'],indexer = pd.factorize(df['category_3'],sort=True)
    df.loc[df['category_3']=='A','category_3_A'] = 1
    df.loc[df['category_3']!='A','category_3_A'] = 0
    df['category_2_raw'] = df['category_2'].fillna(-1)

print('indexer:',indexer)
#indexer is B,C,A so A,B,C becomes 2,0,1

indexer: Index(['A', 'B', 'C'], dtype='object')


In [38]:
# mask1 = historical_transactions['card_id'].isin(['C_ID_4e6213e9bc', 'C_ID_0e171c1b48'])
# # mask1 = historical_transactions['card_id']=='C_ID_0e171c1b48'
# mask2 = historical_transactions['card_id'].isin(['C_ID_5037ff576e', 'C_ID_0e171c1b48'])
# mask3 = historical_transactions['card_id'].isin(['C_ID_5037ff576e', 'C_ID_48fb13e70f'])
# print(historical_transactions.loc[mask1,'category_3_raw'][0:2])
# print(historical_transactions.loc[mask2,'category_3_raw'][0:2])
# print(historical_transactions.loc[mask3,'category_3_raw'][0:2])
# mask1 = new_transactions['card_id'].isin(['C_ID_a97720321f', 'C_ID_fb0875cd28'])
# mask2 = new_transactions['card_id'].isin(['C_ID_a97720321f', 'C_ID_fb0875cd28'])
# mask3 = new_transactions['card_id'].isin(['C_ID_a97720321f', 'C_ID_fb0875cd28'])
# print(new_transactions.loc[mask1,'category_3_raw'][0:2])
# print(new_transactions.loc[mask2,'category_3_raw'][0:2])
# print(new_transactions.loc[mask3,'category_3_raw'][0:2])


In [39]:
print(historical_transactions['category_2_raw'].unique())
mask = historical_transactions['category_2_raw']==-1
print(historical_transactions.loc[mask,'category_2_raw'].astype('float32').sum())
print(historical_transactions.loc[~mask,'category_2_raw'].astype('float32').sum())

[ 1. -1.  3.  5.  2.  4.]
-2652864.0
58067480.0


In [40]:
#get dummies of month_lag
dummies = pd.get_dummies(historical_transactions['month_lag'],prefix='month_lag')
historical_transactions=pd.concat([historical_transactions,dummies],axis=1)
dummies = pd.get_dummies(new_transactions['month_lag'],prefix='month_lag')
new_transactions=pd.concat([new_transactions,dummies],axis=1)
monthlag_dummies_cols = dummies.columns
print(monthlag_dummies_cols)
del dummies;gc.collect()

Index(['month_lag_1', 'month_lag_2'], dtype='object')


91

In [41]:
# historical_transactions.drop(monthlag_dummies_cols,axis=1,inplace=True)
# new_transactions.drop(monthlag_dummies_cols,axis=1,inplace=True)
histmonthlagcols = [col for col in historical_transactions.columns if 'month_lag_' in col]
newmonthlagcols = [col for col in historical_transactions.columns if 'month_lag_' in col]
for col in histmonthlagcols:
    print(historical_transactions[col].describe())

count    2.911236e+07
mean     2.367025e-02
std      1.520197e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: month_lag_-13, dtype: float64
count    2.911236e+07
mean     2.720401e-02
std      1.626775e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: month_lag_-12, dtype: float64
count    2.911236e+07
mean     3.462028e-02
std      1.828161e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: month_lag_-11, dtype: float64
count    2.911236e+07
mean     3.727121e-02
std      1.894256e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: month_lag_-10, dtype: float64
count    2.911236e+07
mean     4.241236e-02
std      2.015281e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+0

In [42]:
for i,df in enumerate([historical_transactions,new_transactions]):
    agg_func = {
        'purchase_amount_scaled' : ['min','max','var','mean','sum'],
        'purchase_amount_per_install': ['min','max','var','mean','sum'],
        'category_2_raw' :['sum','mean'],
        'category_3_raw' :['sum','mean'],
        'category_3_A' :['sum','mean'],
        'city_id' : ['nunique'],
        'state_id' : ['nunique']
    }
    
    monthlagcols = [col for col in df.columns if 'month_lag_' in col]
    for col in monthlagcols:
        agg_func[col] =['sum','mean']
    if (i==0):
        prefix = 'hist'
    else:
        prefix = 'new_hist'
        
    agg_by_card = aggregate_transactions(df,agg_func,'card',prefix)

    print('transaction complete for:',i)    
    train = pd.merge(train, agg_by_card, on='card_id', how='left')
    test = pd.merge(test, agg_by_card, on='card_id', how='left')
    print('merge complete for:',i)

groupby complete
reset index complete
transaction complete for: 0
merge complete for: 0
groupby complete
reset index complete
transaction complete for: 1
merge complete for: 1


Generate Rank Features

In [43]:
def rank_preproc(combined):
    nullzerocols = ['new_hist_purchase_amount_scaled_max','new_hist_purchase_amount_scaled_min',
                   'new_hist_purchase_amount_scaled_sum','new_hist_purchase_amount_scaled_mean',
                'new_hist_purchase_amount_per_install_max','new_hist_purchase_amount_per_install_min',
                'new_hist_purchase_amount_per_install_sum','new_hist_purchase_amount_per_install_mean']
    nullmaxcols = ['new_hist_purchase_date_uptonow','new_hist_first_buy']
    for df in [combined]:
        df['purchase_amount_scaled_total'] = df['new_hist_purchase_amount_scaled_sum']+df['hist_purchase_amount_scaled_sum']
        df['purchase_amount_per_install_total'] = df['new_hist_purchase_amount_per_install_sum']+df['hist_purchase_amount_per_install_sum']
        for col in nullmaxcols:
            maxval = df[col].max() + 30
            df[col].fillna(maxval,inplace=True)
        for col in nullzerocols:
            df[col].fillna(0,inplace=True)
            
    return combined

In [44]:
def gen_newfeats_outlier(combined):
    for df in [combined]:
        df['hist_size_per_merchant'] = df['hist_card_id_size'] / df['hist_merchant_id_nunique']
        df['hist_size_per_merchant_category'] = df['hist_card_id_size'] / df['hist_merchant_category_id_nunique']
        df['hist_size_per_subsector'] = df['hist_card_id_size'] / df['hist_subsector_id_nunique']
        df['hist_size_per_merchant_group'] = df['hist_card_id_size'] / df['hist_merchant_group_id_nunique']
        df['hist_size_per_city'] = df['hist_card_id_size'] / df['hist_city_id_nunique']
    
    return combined
        
def generate_rank_outlier(combined):


        rankdesccols = [
'elapsed_time',
'hist_size_per_merchant',
'new_hist_first_buy',
'new_hist_card_id_size',
'hist_first_buy',
'hist_category_1_mean',
'hist_installments_mean',
'hist_purchase_date_count_mean',
'new_hist_purchase_date_uptonow',
'hist_purchase_date_count_max',
'hist_purchase_date_diff',
'hist_month_lag_-4_sum',
'hist_month_lag_-5_sum',
'hist_month_lag_-6_sum',            
]
    
        rankasccols = [
        'hist_category_2_raw_mean',
        'hist_purchase_amount_per_install_sum',
        'hist_purchase_amount_per_install_mean',
        'hist_card_id_size',
        'new_hist_purchase_date_min',
        'hist_purchase_date_min',
        'hist_month_lag_0_sum',
        'hist_month_lag_-1_sum',
        'hist_month_lag_-2_sum',
        'hist_month_lag_-3_sum',
#         'hist_month_lag_-4_sum',
#         'hist_month_lag_-5_sum',
#         'hist_month_lag_-6_sum',
        'hist_purchase_date_diff',
       # 'hist_most_recent_sales_range_std',
                       
        'hist_authorized_flag_mean',
        'hist_authorized_flag_sum',
        'new_hist_card_id_size',
        'new_hist_purchase_amount_per_install_sum',
        'hist_category_3_A_mean',
        'hist_month_lag_-12_mean',
        'hist_month_lag_-13_sum',
        'new_hist_purchase_date_diff',
        'hist_purchase_amount_per_install_mean',

        ]
        
        rankcols =[col for col in combined.columns if ('rank_' in col)]
        if len(rankcols)!=0:
            combined.drop(rankcols,axis=1,inplace=True)
#         rankcols_train =[col for col in train.columns if ('rank_' in col)]
#         if len(rankcols_train)!=0:
#             train.drop(rankcols_train,axis=1,inplace=True)
#         rankcols_test =[col for col in test.columns if ('rank_' in col)]
#         if len(rankcols_test)!=0:
#             test.drop(rankcols_test,axis=1,inplace=True)
        for i,df in enumerate([combined]):
            print('df :',i)
            for col in rankasccols:
                print('col:',col)
                df['rank_'+col] =  df[col].rank()
    #             df['rank_'+col] = weightasccols[i] * df[col].rank()

            for col in rankdesccols:
                print('col:',col)
                df['rank_'+col] = df[col].rank(ascending=False)
    #             df['rank_'+col] = weightdesccols[i] * df[col].rank(ascending=False)

            rankcols =[col for col in df.columns if ('rank_' in col)
                       & ('hist_purchase_amount_per_install_sum' !=col)
#                        & ('hist_card_id_size' not in col)
                        
                      ]
            df['rank_sum'] = df[rankcols].sum(axis=1) / 1E+5
            df['rank_mean'] = df[rankcols].mean(axis=1) / 1E+5
    #         weight_sum = (np.sum(np.array(weightasccols)) + np.sum(np.array(weightdesccols)))
    #         df['rank_mean'] = df[rankcols].sum(axis=1) / (1E+5 * weight_sum)

    #         df['rank_sum'] = df[rankcols].product(axis=1) / pow(10,len(rankdesccols) + len (rankasccols))

        return combined

In [45]:
def rank_postproc(combined):
    for df in [combined]:
        df['rank_rank_sum']= df['rank_sum'].rank()
        col = 'rank_hist_purchase_amount_per_install_sum'
        df['rank_diff_' + col]= df[col] - df['rank_rank_sum']
    return combined

In [46]:
train['istrain']=1
test['istrain']=0
combined = pd.concat([train,test])

In [47]:
combined = rank_preproc(combined)
print()
print('************ Pre proc complete ********************************')
combined = gen_newfeats_outlier(combined)


************ Pre proc complete ********************************


In [48]:

combined = generate_rank_outlier(combined)
print()
print('************ Generate Rank complete ********************************')
combined = rank_postproc(combined)
print()
print('************ Post proc complete ********************************')

df : 0
col: hist_category_2_raw_mean
col: hist_purchase_amount_per_install_sum
col: hist_purchase_amount_per_install_mean
col: hist_card_id_size
col: new_hist_purchase_date_min
col: hist_purchase_date_min
col: hist_month_lag_0_sum
col: hist_month_lag_-1_sum
col: hist_month_lag_-2_sum
col: hist_month_lag_-3_sum
col: hist_purchase_date_diff
col: hist_authorized_flag_mean
col: hist_authorized_flag_sum
col: new_hist_card_id_size
col: new_hist_purchase_amount_per_install_sum
col: hist_category_3_A_mean
col: hist_month_lag_-12_mean
col: hist_month_lag_-13_sum
col: new_hist_purchase_date_diff
col: hist_purchase_amount_per_install_mean
col: elapsed_time
col: hist_size_per_merchant
col: new_hist_first_buy
col: new_hist_card_id_size
col: hist_first_buy
col: hist_category_1_mean
col: hist_installments_mean
col: hist_purchase_date_count_mean
col: new_hist_purchase_date_uptonow
col: hist_purchase_date_count_max
col: hist_purchase_date_diff
col: hist_month_lag_-4_sum
col: hist_month_lag_-5_sum
col: 

In [49]:
#separate back combined to train and test
train=combined[combined['istrain']==1]
test=combined[combined['istrain']==0]
del combined;gc.collect()

415

In [50]:
target = train['outliers']
target.to_csv('target.csv')

In [51]:
train.to_hdf('train_preproc.hdf',key='data')
test.to_hdf('test_preproc.hdf',key='data')

In [52]:
train.to_csv('train_preproc.csv')
test.to_csv('test_preproc.csv')
print('save csv complete')

save csv complete


In [53]:
%%time
#read and save target encodings
Path = '../input/elo-target-encoding/'
for i in range(5):
    
    file = 'val_targetenc_feats'+str(i)+'.csv'
    val = pd.read_csv(Path+file)
    val.to_csv(file,index=False)
    
    file = 'train_targetenc_feats'+str(i)+'.csv'
    train = pd.read_csv(Path+file)
    train.to_csv(file,index=False)
    
    file = 'test_targetenc_feats'+str(i)+'.csv'
    test = pd.read_csv(Path+file)
    test.to_csv(file,index=False)
    
    print('completed :',i)


completed : 0
completed : 1
completed : 2
completed : 3
completed : 4
CPU times: user 7.53 s, sys: 168 ms, total: 7.7 s
Wall time: 7.75 s


In [54]:
mask = train['target'] < -30
train['outliers'] = 0
train.loc[mask,'outliers'] = 1

KeyError: 'target'